In [1]:

import os
import json
import faiss
import numpy as np
from openai import OpenAI
import traceback
import openai  # 추가


from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API_KEY 값을 가져옴
openai_api_key = os.getenv(OPENAI_API_KEY)
upstage_api_key = os.getenv(UPSATGE_API_KEY)

os.environ["OPENAI_API_KEY"] = openai_api_key

# OpenAI 클라이언트 (Upstage API)
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

# FAISS 인덱스 로드
index = faiss.read_index("knn_index_cosine.faiss")

# chunk-doc 매핑 파일 로드
with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)
    


# Upstage API를 이용한 임베딩 생성
def get_embedding(query_str):
    query_result = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=query_str
    ).data[0].embedding
    return np.array(query_result).reshape(1, -1)

def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

def normalize(embedding):
    return embedding / np.linalg.norm(embedding)

# FAISS에서 벡터 유사도를 이용한 검색
def dense_retrieve(query_str, size=5):
    # 쿼리 임베딩 생성
    query_embedding = get_embedding(query_str)

    # FAISS에서 유사한 k개의 문서 검색
    distances, indices = index.search(query_embedding, size)

    # 검색된 chunk들 가져오기
    retrieved_context = []
    for idx in indices[0]:
        chunk_info = chunk_doc_mapping[idx]
        retrieved_context.append(chunk_info['content'])
    
    return retrieved_context

# OpenAI API 사용 준비 (이미 설정된 api_key와 base_url을 사용)
llm_model = "gpt-3.5-turbo-1106"

# RAG 구현을 위한 LLM 프롬프트 설정
persona_qa = """
## Role: 과학 상식 전문가
## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# LLM과 FAISS 기반 검색을 활용한 RAG 구현
def answer_question(messages):
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}
    
    if not messages:
        print("Error: No input messages provided.")
        return response

    msg = [{"role": "system", "content": persona_qa}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:  # 모든 예외를 처리
        print(f"Error: {e}")
        return response


    # 검색이 필요한 경우 검색 호출 후 결과를 활용
    if "standalone_query" in result.choices[0].message.tool_calls[0].function.arguments:
        query_str = result.choices[0].message.tool_calls[0].function.arguments['standalone_query']
        
        # FAISS를 사용하여 문서 검색
        search_result = dense_retrieve(query_str, 5)

        # 검색 결과를 LLM 답변에 반영
        response["standalone_query"] = query_str
        response["references"] = search_result
        content = json.dumps(search_result)
        messages.append({"role": "assistant", "content": content})

        # 검색된 내용을 기반으로 LLM에 답변 생성 요청
        try:
            qaresult = client.chat.completions.create(
                model=llm_model,
                messages=messages,
                temperature=0,
                seed=1,
                timeout=30
            )
        except Exception as e:
            traceback.print_exc()
            return response

        response["answer"] = qaresult.choices[0].message.content
    return response

# 테스트 데이터로 RAG 실행
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"
messages = [{"role": "user", "content": test_query}]
response = answer_question(messages)

# 결과 출력
print(f"Query: {response['standalone_query']}")
print(f"Answer: {response['answer']}")

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3407819970.py, line 1)

In [19]:

import os
import json
import faiss
import numpy as np
from openai import OpenAI
import traceback

from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API_KEY 값을 가져옴
openai_api_key = os.getenv(OPENAI_API_KEY)
upstage_api_key = os.getenv(UPSATGE_API_KEY)

os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

# FAISS 인덱스 로드
index = faiss.read_index("knn_index_cosine.faiss")

# 문서와 Chunk ID 매핑 파일 로드
with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 코사인 거리 적용을 위해 벡터 정규화 함수 정의
def normalize(embedding):
    return embedding / np.linalg.norm(embedding)

# Upstage API를 이용하여 임베딩 생성
def get_embedding(query_str):
    query_result = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=query_str
    ).data[0].embedding
    query_embedding = np.array(query_result).reshape(1, -1)
    
    # 임베딩을 L2 정규화하여 코사인 거리 적용
    return normalize(query_embedding)

# FAISS를 통한 검색 (KNN 검색)
def dense_retrieve(query_str, size=5):
    # 쿼리 임베딩 생성
    query_embedding = get_embedding(query_str)
    
    # FAISS에서 유사한 k개의 문서 검색 (코사인 거리를 사용)
    distances, indices = index.search(query_embedding, size)

    # 검색된 chunk 정보 가져오기
    retrieved_context = []
    for idx in indices[0]:
        chunk_info = chunk_doc_mapping[idx]
        retrieved_context.append(chunk_info['content'])
    
    return retrieved_context

# OpenAI API 사용 설정
os.environ["OPENAI_API_KEY"] = openai_api_key
llm_model = "gpt-3.5-turbo-1106"
client = OpenAI()

# RAG 구현에 필요한 LLM 프롬프트 설정
persona_qa = """
## Role: 과학 상식 전문가
## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# LLM과 FAISS 기반 검색을 활용한 RAG 구현
def answer_question(messages):
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 LLM 활용
    msg = [{"role": "system", "content": persona_qa}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        print(f"Error: {e}")
        return response

    # 검색이 필요한 경우 FAISS 검색 수행
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # FAISS 검색으로 문서 검색
        search_result = dense_retrieve(standalone_query, 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i, rst in enumerate(search_result):
            retrieved_context.append(rst)
            response["topk"].append(i)
            response["references"].append({"content": rst})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        
        # 검색된 내용을 기반으로 LLM에 답변 생성 요청
        try:
            qaresult = client.chat.completions.create(
                model=llm_model,
                messages=messages,
                temperature=0,
                seed=1,
                timeout=30
            )
        except Exception as e:
            print(f"Error: {e}")
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

# 테스트 데이터로 RAG 실행
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"
messages = [{"role": "user", "content": test_query}]
response = answer_question(messages)

# 결과 출력
print(f"Query: {response['standalone_query']}")
print(f"Answer: {response['answer']}")


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (191897223.py, line 1)

In [4]:
import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용

def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return (embeddings / norms).astype(np.float32)  # float32로 변환

from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API_KEY 값을 가져옴
openai_api_key = os.getenv(OPENAI_API_KEY)
upstage_api_key = os.getenv(OPENAI_API_KEY)

os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

index = faiss.read_index("knn_index_cosine.faiss")

# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 Upstage API를 통해 임베딩 생성
query = "나무의 분류에 대해 조사해 보기 위한 방법은?"
query_result = client.embeddings.create(
    model="solar-embedding-1-large-passage",
    input=query
).data[0].embedding

query_embedding = np.array(query_result).reshape(1, -1)

# 쿼리 임베딩 정규화
normalized_query = normalize(query_embedding)

k = 5
distances, indices = index.search(normalized_query, k)

retrieved_chunks = []
for idx in indices[0]:
    chunk_info = chunk_doc_mapping[idx]
    content = chunk_info['content']
    doc_id = chunk_info['doc_id']
    retrieved_chunks.append((query, content))
    print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
    print(f"연관된 문서 ID: {doc_id}")
    print(f"Chunk 내용: {content}")
    print("-" * 50)
    
os.environ["OPENAI_API_KEY"] = openai_api_key
llm_model = "gpt-3.5-turbo-1106"
client = OpenAI()

def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}
    
    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # 검색 결과 추출
        search_result = get_retrieved_chunks(standalone_query)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i, rst in enumerate(search_result):
            retrieved_context.append(rst[1])
            response["topk"].append(i)
            response["references"].append({"content": rst[1]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                model=llm_model,
                messages=msg,
                temperature=0,
                seed=1,
                timeout=30
            )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    else:
        response["answer"] = result.choices[0].message.content

    return response

# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            if idx >= 3:  # 3번까지만 반복
                break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1
            
            
            
    # 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/ir/data/eval.jsonl", "sample_submission.csv")

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3887020913.py, line 1)

In [5]:

import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용


from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API_KEY 값을 가져옴
openai_api_key = os.getenv(OPENAI_API_KEY)
upstage_api_key = os.getenv(OPENAI_API_KEY)

os.environ["OPENAI_API_KEY"] = openai_api_key

def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return (embeddings / norms).astype(np.float32)  # float32로 변환

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

index = faiss.read_index("knn_index_cosine.faiss")

# 문서 ID 및 chunk ID 매핑 파일 로드
with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 Upstage API를 통해 임베딩 생성
def get_retrieved_chunks(query, k=5):
    query_result = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=query
    ).data[0].embedding

    query_embedding = np.array(query_result).reshape(1, -1)

    # 쿼리 임베딩 정규화
    normalized_query = normalize(query_embedding)

    # FAISS 검색 실행
    distances, indices = index.search(normalized_query, k)

    retrieved_chunks = []
    topk = []
    references = []

    for idx, distance in zip(indices[0], distances[0]):
        chunk_info = chunk_doc_mapping[idx]
        content = chunk_info['content']
        doc_id = chunk_info['doc_id']
        topk.append(doc_id)
        references.append({
            "score": float(distance),  # float32를 float으로 변환
            "content": content
        })
        print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
        print(f"연관된 문서 ID: {doc_id}")
        print(f"Chunk 내용: {content}")
        print("-" * 50)

    return topk, references

# LLM을 통한 RAG 시스템 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}
    
    # 메시지에서 검색 쿼리를 추출
    query = messages[0]['content']

    # standalone_query로 설정
    response["standalone_query"] = query

    # 검색 결과 추출
    topk, references = get_retrieved_chunks(query)

    # 검색 결과를 response에 추가
    response["topk"] = topk
    response["references"] = references

    # 임의로 생성된 답변 (LLM 통합 부분에 적용)
    response["answer"] = f"{query}에 대한 결과는 위 문서들을 참조하세요."

    return response

# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx >= 3:  # 3번까지만 반복
            #     break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/ir/data/eval.jsonl", "/ir/sample_submission_2.csv")


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (346362905.py, line 1)

In [6]:

import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용

from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API_KEY 값을 가져옴
openai_api_key = os.getenv(OPENAI_API_KEY)
upstage_api_key = os.getenv(OPENAI_API_KEY)

os.environ["OPENAI_API_KEY"] = openai_api_key

def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return (embeddings / norms).astype(np.float32)  # float32로 변환

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

index = faiss.read_index("knn_index_cosine.faiss")

# 문서 ID 및 chunk ID 매핑 파일 로드
with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 Upstage API를 통해 임베딩 생성
def get_retrieved_chunks(query, k=5):
    query_result = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=query
    ).data[0].embedding

    query_embedding = np.array(query_result).reshape(1, -1)

    # 쿼리 임베딩 정규화
    normalized_query = normalize(query_embedding)

    # FAISS 검색 실행
    distances, indices = index.search(normalized_query, k)

    retrieved_chunks = []
    topk = []
    references = []

    for idx, distance in zip(indices[0], distances[0]):
        chunk_info = chunk_doc_mapping[idx]
        content = chunk_info['content']
        doc_id = chunk_info['doc_id']
        topk.append(doc_id)
        references.append({
            "score": float(distance),  # float32를 float으로 변환
            "content": content
        })
        print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
        print(f"연관된 문서 ID: {doc_id}")
        print(f"Chunk 내용: {content}")
        print("-" * 50)

    return topk, references

# LLM을 통한 질문 판별 함수
def requires_retrieval(messages):
    {
    "role": "system",
    "content": """
    당신은 과학적 질문과 비과학적 질문을 구분하는 전문가입니다. 사용자가 과학, 물리, 화학, 생물학, 의학 또는 기술과 관련된 질문을 하면, 검색이 필요하다고 표시하세요. 질문이 더 일반적인 내용이라면 직접 답변을 생성하세요.
    """
    }
    
    # LLM에게 대화가 과학적인지 판단하도록 요청
    result = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[system_message] + messages,
        temperature=0
    )

    # LLM이 판단한 결과 반환 (True: 검색 필요, False: 검색 불필요)
    return "retrieval" in result.choices[0].message["content"].lower()

# 과학 상식 전문가 프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# 과학 상식 외 질문에 대한 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# LLM을 통한 RAG 시스템 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    if requires_retrieval(messages):
        # 과학 관련 질문의 경우
        query = " ".join([msg['content'] for msg in messages if msg['role'] == 'user'])
        print(f"Transformed Query for Retrieval: {query}")

        # standalone_query로 설정
        response["standalone_query"] = query

        # 검색 결과 추출
        topk, references = get_retrieved_chunks(query)

        # 검색 결과를 response에 추가
        response["topk"] = topk
        response["references"] = references

        # 검색된 문서에서 답변 생성 (프롬프트 적용)
        content = "\n".join([ref["content"] for ref in references])
        messages.append({"role": "assistant", "content": content})

        result = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0
        )
        response["answer"] = result.choices[0].message["content"]
    
    else:
        # 과학과 관련 없는 질문에 대해 LLM이 직접 답변 생성
        messages.append({"role": "system", "content": persona_function_calling})

        result = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0
        )
        response["answer"] = result.choices[0].message["content"]

    return response

# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            if idx >= 3:  # 3번까지만 반복
                break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/ir/data/eval.jsonl", "/ir/sample_submission.csv")


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (853420656.py, line 1)

In [7]:

import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용

from dotenv import load_dotenv

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return (embeddings / norms).astype(np.float32)  # float32로 변환

# Upstage API 클라이언트 설정

index = faiss.read_index("knn_index_cosine.faiss")

# 문서 ID 및 chunk ID 매핑 파일 로드
with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 Upstage API를 통해 임베딩 생성
def get_retrieved_chunks(query, k=5):
    query_result = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=query
    ).data[0].embedding

    query_embedding = np.array(query_result).reshape(1, -1)

    # 쿼리 임베딩 정규화
    normalized_query = normalize(query_embedding)

    # FAISS 검색 실행
    distances, indices = index.search(normalized_query, k)

    retrieved_chunks = []
    topk = []
    references = []

    for idx, distance in zip(indices[0], distances[0]):
        chunk_info = chunk_doc_mapping[idx]
        content = chunk_info['content']
        doc_id = chunk_info['doc_id']
        topk.append(doc_id)
        references.append({
            "score": float(distance),  # float32를 float으로 변환
            "content": content
        })
        print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
        print(f"연관된 문서 ID: {doc_id}")
        print(f"Chunk 내용: {content}")
        print("-" * 50)

    return topk, references

# LLM을 통한 질문 판별 함수
def requires_retrieval(messages):
    # LLM에게 과학적인 질문인지 판단하도록 요청
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 질문과 비과학적 질문을 구분하는 전문가입니다. 사용자가 과학, 물리, 화학, 생물학, 의학 또는 기술과 관련된 질문을 하면, 검색이 필요하다고 표시하세요. 질문이 더 일반적인 내용이라면 직접 답변을 생성하세요.
        """
    }
    
    try:
        result = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[system_message] + messages,
            temperature=0
        )
    except openai.error.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return response

    # LLM이 판단한 결과 반환 (True: 검색 필요, False: 검색 불필요)
    return "retrieval" in result.choices[0].message["content"].lower()

# 과학 상식 전문가 프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# 과학 상식 외 질문에 대한 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# LLM을 통한 RAG 시스템 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    if requires_retrieval(messages):
        # 과학 관련 질문의 경우
        query = " ".join([msg['content'] for msg in messages if msg['role'] == 'user'])
        print(f"Transformed Query for Retrieval: {query}")

        # standalone_query로 설정
        response["standalone_query"] = query

        # 검색 결과 추출
        topk, references = get_retrieved_chunks(query)

        # 검색 결과를 response에 추가
        response["topk"] = topk
        response["references"] = references

        # 검색된 문서에서 답변 생성 (프롬프트 적용)
        content = "\n".join([ref["content"] for ref in references])
        messages.append({"role": "assistant", "content": content})

        result = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0
        )
        response["answer"] = result.choices[0].message["content"]
    
    else:
        # 과학과 관련 없는 질문에 대해 LLM이 직접 답변 생성
        messages.append({"role": "system", "content": persona_function_calling})

        result = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0
        )
        response["answer"] = result.choices[0].message["content"]

    return response

# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            if idx >= 3:  # 3번까지만 반복
                break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/ir/data/eval.jsonl", "/ir/sample_submission.csv")


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (2303401404.py, line 1)

In [6]:



import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용
import os

In [9]:


from dotenv import load_dotenv

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)
os.environ["OPENAI_API_KEY"] = openai_api_key
llm_model = "gpt-3.5-turbo-1106"
client = OpenAI()

# openai.api_key = os.getenv("OPENAI_API_KEY")

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3474105294.py, line 1)

In [10]:

def transform_query_with_llm(messages):
    """
    LLM을 사용하여 사용자의 여러 메시지를 기반으로 검색에 적합한 단일 쿼리 생성.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 사용자의 여러 대화 메시지를 하나의 검색 쿼리로 변환하는 전문가입니다. 
        사용자의 대화 중 중요한 정보만을 추출하여 간결하고 명확한 검색용 쿼리로 변환하세요. 
        대화 형식이 아닌, 연구나 조사를 위한 검색어처럼 정확한 질문을 생성하세요.
        """
    }

    # 사용자 메시지 준비
    dialogue_messages = [{"role": msg['role'], "content": msg['content']} for msg in messages]

    # LLM 호출을 위한 메시지 배열 생성
    full_message = [system_message] + dialogue_messages

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client.chat.completions.create(
        model="gpt-4o",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 쿼리 반환 (ChatCompletionMessage 형식에서 content에 직접 접근)
    transformed_query = result.choices[0].message.content
    print(f"변환된 쿼리: {transformed_query}")
    return transformed_query


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (1256807387.py, line 1)

In [179]:
messages = [
    {"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."},
    {"role": "assistant", "content": "네 맞습니다."},
    {"role": "user", "content": "원인이 뭘까."}
]


3

In [76]:
messages = [
{"role": "user", "content": "세균이 나쁜줄말 알았는데 그게 아니야?"},
{"role": "assistant", "content": "네 맞습니다."},
{"role": "user", "content": "그럼 순기능에 대해 알려줄래?"}
]

In [94]:
messages = [
    
{"role": "user", "content": "은하에는 엄청나게 많은 별들이 모여 있잖아?"}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "그 많은 별들이 어떻게 뭉쳐 있을 수 있지?"}


]



In [95]:
transform_query_with_llm(messages)

변환된 쿼리: 은하 내 별들이 중력으로 어떻게 모여 있는가?


'은하 내 별들이 중력으로 어떻게 모여 있는가?'

In [97]:
def science_query_detection(message):
    """
    LLM을 사용하여 단일 사용자의 질문을 검색에 적합한 쿼리로 변환.
    질문이 과학과 관련되어 있으면 '과학 관련 질문입니다'라고 답하고, 
    그렇지 않으면 '과학 관련 질문이 아닙니다'라고 답변.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 질문과 비과학적 질문을 구분하는 전문가입니다. 
        질문이 과학, 물리, 화학, 생물학, 의학 또는 기술과 관련되어 있으면 '과학 관련 질문입니다'라고 답하세요. 
        그렇지 않으면 '과학 관련 질문이 아닙니다'라고 답하세요. 
        과학 관련 질문일 경우에는 좀 더 명확하고 간결한 검색용 질문으로 변환하세요.
        """
    }

    # 사용자 질문 준비
    user_message = {"role": message['role'], "content": message['content']}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, user_message]

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client.chat.completions.create(
        model="gpt-3.5-turbo",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 쿼리 반환 (ChatCompletionMessage 형식에서 content에 직접 접근)
    llm_response = result['choices'][0]['message']['content']

    # 과학 관련 질문 여부 판단 및 처리
    if "과학 관련 질문이 아닙니다" in llm_response:
        print("과학 관련 질문이 아닙니다.")
        return "과학 관련 질문이 아닙니다."
    else:
        # 과학 관련 질문일 경우 변환된 쿼리 반환
        refined_query = llm_response.strip()
        print(f"변환된 쿼리: {refined_query}")
        return refined_query

In [99]:
message = [
    {"role": "user", "content": "바다가 육지의 날씨에 미치는 영향은?"},
]

In [100]:
science_query_detection(message)

TypeError: list indices must be integers or slices, not str

In [103]:
def science_query_detection(message):
    """
    LLM을 사용하여 사용자의 질문이 과학과 관련된지 여부를 판단하고, 
    관련된 경우 검색용으로 적합한 질문으로 변환.
    질문이 과학과 관련되어 있으면 '과학 관련 질문입니다'라고 답하고, 
    그렇지 않으면 '과학 관련 질문이 아닙니다'라고 답변.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 질문과 비과학적 질문을 구분하는 전문가입니다. 
        질문이 과학, 물리, 화학, 생물학, 의학, 영양학, 인간의성, 또는 기술과 관련되어 있으면 '과학 관련 질문입니다'라고 답하세요. 
        그렇지 않으면 '과학 관련 질문이 아닙니다'라고 답하세요. 
        과학 관련 질문일 경우에는 좀 더 명확하고 간결한 검색용 질문으로 변환하세요.
        """
    }

    # 단일 질문을 message로 받아 처리
    user_message = {"role": message[0]['role'], "content": message[0]['content']}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, user_message]

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client.chat.completions.create(
        model="gpt-4o",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 반환
    llm_response = result['choices'][0]['message']['content']

    # 과학 관련 질문 여부 판단 및 처리
    if "과학 관련 질문이 아닙니다" in llm_response:
        print("과학 관련 질문이 아닙니다.")
        return "과학 관련 질문이 아닙니다."
    else:
        # 과학 관련 질문일 경우 변환된 쿼리 반환
        refined_query = llm_response.strip()
        print(f"변환된 쿼리: {refined_query}")
        return refined_query

In [121]:
def science_query_detection(message):
    """
    LLM을 사용하여 단일 사용자의 질문을 검색에 적합한 쿼리로 변환.
    질문이 과학과 관련되어 있으면 '과학 관련 질문입니다'라고 답하고, 
    그렇지 않으면 '과학 관련 질문이 아닙니다'라고 답변.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 질문과 비과학적 질문을 구분하는 전문가입니다. 
        질문이 물리학, 화학, 생물학, 임상 의학, 바이러스학, 인간 노화, 컴퓨터 공학, 해부학, 유전학, 전기 공학, 천문학, 세계 상식 또는 기술과 관련되지 않으면 '과학 관련 질문이 아닙니다'라고 답하세요.
        과학 관련 질문일 경우에는 자료 검색에 적합하도록 명확하고 간결한 질문으로 변환하세요.
        """
    }

    # 사용자 질문 준비
    user_message = {"role": message[0]['role'], "content": message[0]['content']}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, user_message]

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client.chat.completions.create(
        model="gpt-4o",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 반환
    llm_response = result.choices[0].message.content

    # 과학 관련 질문 여부 판단 및 처리
    if "과학 관련 질문이 아닙니다" in llm_response:
        print("과학 관련 질문이 아닙니다.")
        return "과학 관련 질문이 아닙니다."
    else:
        # 과학 관련 질문일 경우 변환된 쿼리 반환
        refined_query = llm_response.strip()
        print(f"변환된 쿼리: {refined_query}")
        return refined_query


In [127]:
message = [
    {"role": "user", "content": "나 과학 공부하기 싫어"}
]

science_query_detection(message)

과학 관련 질문이 아닙니다.


'과학 관련 질문이 아닙니다.'

In [ ]:
def question_maker(document):
    """
    LLM을 사용하여 문서에서 질문을 만들어낸다. 문서가 상식적이거나 논리적이지 않으면 말이 되지 않는다고 말한다.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 문서를 보고 질문을 만들어 내는 전문가 입니다
        질문이 상식적이지 않거나 논리적이지 않으면 말이 되지 않습니다. 라고 말하고
        아니면 이 문서에서 답을 찾을 수 있을만한 질문 5개를 만들어 내시오.
        """
    }

    # 사용자 질문 준비
    document = {"src": document[0]['src'], "content": document[0]['content']}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, user_message]

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client.chat.completions.create(
        model="gpt-4o",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 반환
    llm_response = result.choices[0].document.content

    # 과학 관련 질문 여부 판단 및 처리
    if "과학 관련 질문이 아닙니다" in llm_response:
        print("과학 관련 질문이 아닙니다.")
        return "과학 관련 질문이 아닙니다."
    else:
        # 과학 관련 질문일 경우 변환된 쿼리 반환
        refined_query = llm_response.strip()
        print(f"변환된 쿼리: {refined_query}")
        return refined_query


In [9]:
# def question_maker(document):
#     """
#     LLM을 사용하여 문서에서 논리적이고 상식적인 질문 5개를 생성합니다.
#     문서가 논리적이지 않거나 상식적이지 않으면 "말이 되지 않습니다."라는 응답을 반환합니다.
#     """
#     # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
#     system_message = {
#         "role": "system",
#         "content": """
#         당신은 과학적 문서를 보고 논리적이고 상식적인 질문을 만들어내는 전문가입니다.
#         문서가 비논리적이거나 상식적이지 않다면 "말이 되지 않습니다."라고 답변하세요.
#         그렇지 않다면, 이 문서에서 답을 찾을 수 있을만한 5개의 질문을 논리적이고 명확하게 작성하세요.
#         """
#     }

#     # 문서의 src와 content를 포함한 메시지 준비
#     document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}

#     # LLM 호출을 위한 전체 메시지 배열 생성
#     full_message = [system_message, document_message]

#     # OpenAI API 호출하여 적절한 질문 생성
#     result = client.chat.completions.create(
#         model="gpt-4",  # 사용할 모델 지정
#         messages=full_message,
#         temperature=0
#     )

#     # LLM이 생성한 응답을 가져오기
#     llm_response = result.choices[0].message["content"]

#     # 과학적 질문을 반환하거나, 문서가 비논리적인 경우 처리
#     if "말이 되지 않습니다" in llm_response:
#         print("문서가 논리적이지 않습니다.")
#         return "말이 되지 않습니다."
#     else:
#         # 논리적인 질문 5개를 반환
#         questions = llm_response.strip().split("\n")
#         print(f"생성된 질문들: {questions}")
#         return questions


# def question_maker(document):
#     """
#     LLM을 사용하여 문서에서 논리적이고 상식적인 질문 5개를 생성합니다.
#     문서가 논리적이지 않거나 상식적이지 않으면 "말이 되지 않습니다."라는 응답을 반환합니다.
#     """
#     # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
#     system_message = {
#         "role": "system",
#         "content": """
#         당신은 과학적 문서를 분석하여 그 문서에서 논리적이고 상식적인 질문을 만드는 전문가입니다.
#         문서가 비논리적이거나 상식적이지 않다면, "이 문서는 논리적이지 않습니다."라고 답변하세요.
#         논리적이고 상식적인 문서라면, 그 문서에서 답을 찾을 수 있을 만한 명확하고 중요한 5개의 질문을 생성하세요.
#         질문은 반드시 문서의 중요한 개념이나 정보를 기반으로 하며, 과학적 사실이나 원리를 반영해야 합니다.
#         질문은 "왜", "어떻게", "무엇을" 등으로 시작하는 형태로 구성해야 합니다.
#         질문을 나열할 때는 번호를 매겨 명확히 구분하고, 답을 유도하지 않도록 주의하세요.
#         """
#     }

#     # 문서의 src와 content를 포함한 메시지 준비
#     document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}

#     # LLM 호출을 위한 전체 메시지 배열 생성
#     full_message = [system_message, document_message]

#     # OpenAI API 호출하여 적절한 질문 생성
#     result = client.chat.completions.create(
#         model="gpt-4o",  # 사용할 모델 지정
#         messages=full_message,
#         temperature=0
#     )

#     # LLM이 생성한 응답을 가져오기
#     llm_response = result.choices[0].message.content

#     # 문서가 비논리적인 경우 처리
#     if "이 문서는 논리적이지 않습니다" in llm_response:
#         print("문서가 논리적이지 않습니다.")
#         return "이 문서는 논리적이지 않습니다."
#     else:
#         # 논리적인 질문 5개를 반환
#         questions = llm_response.strip().split("\n")
#         print(f"생성된 질문들: {questions}")
#         return questions
    
    
def question_maker(document):
    """
    LLM을 사용하여 문서에서 논리적 오류와 상식적 타당성을 평가하고, 
    두 가지 조건이 모두 충족될 경우 논리적이고 상식적인 질문 5개를 생성합니다.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 문서를 평가하는 전문가입니다.

        **1. 논리적 오류 평가**:
            - 문서의 설명에서 과학적 원리와 모순되거나 자명한 논리적 오류가 있는지 확인하세요.
            - 예를 들어, '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은, 과학적 원리(열역학)와는 일치할지 모르지만 그 본래 목적과는 어긋납니다.

        **2. 상식적 타당성 평가**:
            - 설명이 과학적으로 맞다고 해도 상식적으로 그 설명이 현실적인지 판단하세요.
            - 예를 들어, '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않으며, 주된 용도와는 부합하지 않습니다.

        **결과**:
            - 두 가지 조건이 모두 충족되지 않으면 "이 문서는 논리적 오류가 있으며, 상식적이지 않습니다."라고 답변하세요.
            - 두 가지 조건이 모두 충족되면, 이 문서에서 답을 찾을 수 있을 만한 논리적이고 상식적인 질문 5개를 생성하세요.
        """
    }

    # 사용자 문서 입력 준비
    document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, document_message]

    # OpenAI API 호출하여 적절한 질문 생성
    result = client.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 가져오기
    llm_response = result.choices[0].message.content

    # 논리적 오류와 상식적 타당성을 반환하거나, 논리적인 질문 5개를 반환
    if "논리적 오류가 있으며, 상식적이지 않습니다" in llm_response:
        print("문서가 논리적 오류가 있으며, 상식적이지 않습니다.")
        return "이 문서는 논리적 오류가 있으며, 상식적이지 않습니다."
    else:
        # 논리적인 질문 5개를 반환
        questions = llm_response.strip().split("\n")
        print(f"생성된 질문들: {questions}")
        return questions


{"docid": "82b095fd-2fb6-48ae-8476-2f457f8b6650", "src": "ko_ai2_arc__ARC_Challenge__train", "content": "인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다."}


In [12]:
document = [
    
{"docid": "82b095fd-2fb6-48ae-8476-2f457f8b6650", "src": "ko_ai2_arc__ARC_Challenge__train", "content": "인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다."}

]

question_maker(document)

문서가 논리적 오류가 있으며, 상식적이지 않습니다.


'이 문서는 논리적 오류가 있으며, 상식적이지 않습니다.'

'ko_mmlu__nutrition__test'

In [152]:
document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}
document_message

{'role': 'user',
 'content': '문서 출처: ko_mmlu__nutrition__test\n내용: 건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.'}

{'role': 'user',
 'content': '문서 출처: ko_mmlu__nutrition__test\n내용: 건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.'}